In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
from ultralytics.utils.patches import imread
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ImageOps
import torch

In [2]:
from glob import glob
latest_run_dir = sorted(glob("zindi_challenge_cacao_stage2/train*"), key=lambda x: int(x.split('train')[-1]))[-1]

# Validate the model on the validation set
BEST_PATH = f"{latest_run_dir}/weights/best.pt"
# BEST_PATH = 'zindi_challenge_cacao_stage2/train10/last2.pt'
BEST_PATH

'zindi_challenge_cacao_stage2/train10/weights/best.pt'

In [3]:
# Validate the model on the validation set
BEST_CFG = f"{latest_run_dir}/args.yaml"
# BEST_CFG = 'zindi_challenge_cacao_stage2/train6/args.yaml'
BEST_CFG

'zindi_challenge_cacao_stage2/train10/args.yaml'

In [4]:
# Load the trained YOLO model
model = YOLO(BEST_PATH, task="detect").eval()
model.args

{'task': 'detect',
 'data': 'data.yaml',
 'imgsz': 1024,
 'single_cls': False,
 'model': 'zindi_challenge_cacao_stage2/train10/weights/best.pt'}

In [5]:
with open(BEST_CFG, 'r') as f:
	cfg: dict = yaml.safe_load(f)

# Batch size for predictions
batch_size = 16

cfg["device"] = "cuda"
cfg["batch"] = batch_size
cfg["conf"] = 0.0
cfg["verbose"] = False
cfg["nms"] = True
cfg["iou"] = .6
cfg["agnostic_nms"] = False

cfg.pop("source", None)
# cfg.pop("batch_size")
cfg.pop("visualize", None)
# cfg.pop("data", None)
cfg.pop("name", None)
# cfg.pop("half", None)

cfg.pop("model", None)

keys = list(cfg.keys())
for col in keys:
    if (
        "show" in col  # Existing: removes show, show_labels, show_conf, show_boxes
        or "save" in col  # Existing: removes save, save_period, save_json, save_frames, save_txt, save_conf, save_crop, save_dir
        # or "freeze" in col  # Existing
        # Consider `col == 'nms'` instead of `"nms" in col` to avoid removing `agnostic_nms`
        # `agnostic_nms` is often useful for prediction.
        # or col == 'nms' # Removes the general nms flag if present
        # or "multi_scale" in col  # Existing
        or "plot" in col  # Existing
        # or "aug" in col  # Existing: removes augment, auto_augment. Also consider removing individual aug params if TTA is off.
        or "drop" in col  # Existing
        # or "iou" in col  # Existing: removes training iou. Prediction uses its own iou parameter.
        or "lr" in col  # Existing: removes lr0, lrf, cos_lr, warmup_bias_lr
        or "mom" in col  # Existing: removes momentum, warmup_momentum
        or "wei" in col  # Existing: removes weight_decay
        # The 'half' parameter is crucial for mixed-precision inference.
        # If cfg['half'] is intended for prediction, this condition should not remove it.
        or "half" in col # Existing: Problematic if 'half' is needed for prediction.
        or "nbs" in col  # Existing
        # New conditions:
        or "epoch" in col  # Removes epochs, warmup_epochs
        or col == 'optimizer'
        # or "worker" in col  # Removes workers
        # or col == 'val' or col == 'split' # Removes validation config from training
        or col == 'project' # Removes experiment project name
        # or col in ['box', 'cls', 'dfl', 'pose', 'kobj']  # Removes loss component weights
        # or col in ['format', 'keras', 'simplify', 'opset', 'int8', 'dynamic', 'workspace'] # Removes export-related params
        or col == 'patience'
        or col == 'cache'
        # or col == 'seed'
        or col == 'rect' # Rectangular training
        or col == 'resume'
        or col == 'amp' # Training AMP flag (prediction uses 'half')
        or col == 'profile'
        or col == 'tracker'
        or col == 'task'
        or col == 'mode' # e.g., mode: train
        or col == 'pretrained'
        # or col == 'deterministic'
        or col == 'exist_ok'
        # or col == 'single_cls'
        or col == 'time' # training time limit
        or col == 'cfg' # path to model cfg yaml (e.g., yolov8n.yaml)
        # If 'augment' key is removed (disabling Test Time Augmentation),
        # you might also want to remove individual augmentation parameters:
        # or col in ['degrees', 'translate', 'scale', 'shear', 'perspective', 'flipud', 'fliplr', 'bgr', 'mosaic', 'mixup', 'cutmix', 'copy_paste', 'erasing']
        # or col.startswith('hsv_') # hsv_h, hsv_s, hsv_v
        # or col == "conf"
    ):
        cfg.pop(col)

cfg["save_dir"] = "runs/detect/val"
print(cfg)

{'data': 'data.yaml', 'batch': 16, 'imgsz': 1024, 'device': 'cuda', 'workers': 4, 'verbose': False, 'seed': 0, 'deterministic': False, 'single_cls': False, 'close_mosaic': 10, 'fraction': 1.0, 'freeze': None, 'multi_scale': True, 'overlap_mask': True, 'mask_ratio': 4, 'val': True, 'split': 'val', 'conf': 0.0, 'iou': 0.6, 'max_det': 150, 'dnn': False, 'vid_stride': 1, 'stream_buffer': False, 'augment': True, 'agnostic_nms': False, 'classes': None, 'retina_masks': False, 'embed': None, 'line_width': None, 'format': 'torchscript', 'keras': False, 'optimize': False, 'int8': False, 'dynamic': False, 'simplify': True, 'opset': None, 'workspace': None, 'nms': True, 'box': 7.5, 'cls': 1.0, 'dfl': 1.5, 'pose': 12.0, 'kobj': 1.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.3, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.1, 'cutmix': 0.0, 'copy_paste': 0.1, 'copy_paste_mode': 'mixup', 'auto_augment': 'augmix', 

In [6]:
metrics = model.val(**cfg) # **cfg

Ultralytics 8.3.130 🚀 Python-3.10.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48577MiB)
YOLO11m summary (fused): 125 layers, 20,032,345 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3333.0±1530.7 MB/s, size: 1712.0 KB)


val: Scanning /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/dataset/labels/val.cache... 277 images, 0 backgrounds, 0 corrupt: 100%|██████████| 277/277 [00:00<?, ?it/s]

val: /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/dataset/images/val/ID_DNHsfS.jpg: corrupt JPEG restored and saved
val: /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/dataset/images/val/ID_DOTVrd.jpg: corrupt JPEG restored and saved
val: /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/dataset/images/val/ID_ET34jY.jpeg: corrupt JPEG restored and saved
val: /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/dataset/images/val/ID_FBviFm.jpg: corrupt JPEG restored and saved
val: /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/dataset/images/val/ID_FH5WPo.jpg: corrupt JPEG restored and saved
val: /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/dataset/images/val/ID_FtYRqz.jpeg: corrupt JPEG restored and saved
val: /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/18 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:35<00:00,  1.96s/it]


                   all        277        498      0.885      0.692      0.829      0.572
Speed: 1.8ms preprocess, 21.8ms inference, 0.0ms loss, 94.0ms postprocess per image
Results saved to /data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/runs/detect/val7


In [7]:
model.args

{'task': 'detect',
 'data': 'data.yaml',
 'imgsz': 1024,
 'single_cls': False,
 'model': 'zindi_challenge_cacao_stage2/train10/weights/best.pt'}

In [8]:
model.overrides

{'task': 'detect',
 'data': 'data.yaml',
 'imgsz': 1024,
 'single_cls': False,
 'model': 'zindi_challenge_cacao_stage2/train10/weights/best.pt'}